In [210]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from bayes_opt import BayesianOptimization
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MaxAbsScaler
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from prince import FAMD
from scipy import sparse
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

In [2]:
def encoder(df, col_split):
    """
    Prepares the data for the model by standardizing the continuous features, 
    converting the categorical features to strings, 
    and converting sparse columns to a dense format for FAMD.
    """
    categorical_cols = df.columns[:col_split]

    # converting the categorical features to strings
    df[categorical_cols] = df[categorical_cols].astype(str)

    return df

def load_data(data_address='beta_dates/beta_data_7_60.csv', label_address='beta_dates/true_labels.csv'):
    df = pd.read_csv(data_address, index_col=0)
    y = pd.read_csv(label_address, index_col=0).values[:,0]
    y = y + 1
    X = encoder(df, 4)
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

    return X_train, X_test, y_train, y_test
def load_data_cv(data_address='beta_dates/beta_data_7_60.csv', label_address='beta_dates/true_labels.csv'):
    df = pd.read_csv(data_address, index_col=0)
    y = pd.read_csv(label_address, index_col=0).values[:,0]
    y = y + 1
    X = encoder(df, 4)

    return X, y

In [4]:
params = {'rf__max_depth': 3, 'rf__max_features': 0.6639830714160453, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 8, 'rf__n_estimators': 143}

In [225]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from prince import FAMD  # Ensure prince is installed
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import StratifiedKFold
import numpy as np

# Number of splits for K-Fold Cross-Validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True)

# Load your data
X, y = load_data_cv('beta_dates/beta_data_2_42.csv', 'beta_dates/true_labels.csv')

# Perform FAMD on the dataset
famd = FAMD(n_components=8)
famd.fit(X)
X = famd.transform(X)

# Create a pipeline with the best parameters
pipe = Pipeline([
    ('scaler1', MaxAbsScaler()),
    ('scaler2', MaxAbsScaler()),  # Consider if you really need two MaxAbsScalers
    ('standard_scaler', StandardScaler()),
    ('classifier', RandomForestClassifier(bootstrap=False, criterion="gini", max_features=0.55, min_samples_leaf=6, min_samples_split=17, n_estimators=100))
])

# Perform K-Fold Cross-Validation
accuracy_scores = []
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Fit the pipeline to the training data
    pipe.fit(X_train, y_train)

    # Make predictions on the test set
    predictions = pipe.predict(X_test)

    # Evaluate the model
    print(classification_report(y_test, predictions))
    print("Accuracy:", accuracy_score(y_test, predictions))
    accuracy_scores.append(accuracy_score(y_test, predictions))

print("Average Accuracy:", np.mean(accuracy_scores))


              precision    recall  f1-score   support

         0.0       0.33      0.14      0.20         7
         1.0       0.74      0.92      0.82        38
         2.0       0.80      0.40      0.53        10

    accuracy                           0.73        55
   macro avg       0.63      0.49      0.52        55
weighted avg       0.70      0.73      0.69        55

Accuracy: 0.7272727272727273
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         6
         1.0       0.73      0.95      0.83        38
         2.0       0.80      0.36      0.50        11

    accuracy                           0.73        55
   macro avg       0.51      0.44      0.44        55
weighted avg       0.67      0.73      0.67        55

Accuracy: 0.7272727272727273
              precision    recall  f1-score   support

         0.0       1.00      0.33      0.50         6
         1.0       0.81      0.92      0.86        38
         2.0     

In [213]:
# Number of splits for K-Fold Cross-Validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True)

X, y = load_data_cv('beta_dates/beta_data_2_42.csv', 'beta_dates/true_labels.csv')

famd = FAMD(n_components=8)
famd.fit(X)
X = famd.transform(X)

# create a pipeline with the best parameters
pipe = Pipeline([
    ('rf', RandomForestClassifier()),
])

pipe.set_params(**params)

# Perform K-Fold Cross-Validation

accuracy_scores = []
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Fit the pipeline to the training data
    pipe.fit(X_train, y_train)

    # Make predictions on the test set
    predictions = pipe.predict(X_test)

    # Evaluate the model
    print(classification_report(y_test, predictions))
    print("Accuracy:", accuracy_score(y_test, predictions))
    accuracy_scores.append(accuracy_score(y_test, predictions))

print("Average Accuracy:", np.mean(accuracy_scores))

NameError: name 'params' is not defined

In [26]:
X, y = load_data_cv('beta_dates/beta_data_2_42.csv', 'beta_dates/true_labels.csv')

# Original features plus the case of dropping no feature
original_features = X.columns.tolist()
original_features.append(None)  # Represents the case of dropping no feature

# Initialize Stratified K-Fold
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True)

# Store average accuracies for each feature drop scenario
feature_drop_accuracies = {}

for drop_feature in original_features:
    iteration_accuracies = []

    for iteration in range(20):
        # Drop one feature for this iteration, if specified
        X_dropped = X.drop(columns=[drop_feature]) if drop_feature is not None else X

        # Perform FAMD on the modified dataset
        famd = FAMD(n_components=8)
        famd.fit(X_dropped)
        X_famd = famd.transform(X_dropped)

        # Create a pipeline
        pipe = Pipeline([
            ('rf', RandomForestClassifier()),  # Assuming params are set for RandomForest
        ])

        pipe.set_params(**params)

        accuracy_scores = []
        for train_index, test_index in skf.split(X_famd, y):
            X_train, X_test = X_famd.iloc[train_index], X_famd.iloc[test_index]
            y_train, y_test = y[train_index], y[test_index]

            # Fit the pipeline to the training data
            pipe.fit(X_train, y_train)

            # Make predictions on the test set
            predictions = pipe.predict(X_test)

            # Evaluate the model
            accuracy_scores.append(accuracy_score(y_test, predictions))

        # Calculate average accuracy for this iteration
        iteration_accuracies.append(np.mean(accuracy_scores))

    # Calculate the overall average accuracy after dropping the feature
    avg_accuracy = np.mean(iteration_accuracies)
    feature_name = drop_feature if drop_feature is not None else "No Feature Dropped"
    feature_drop_accuracies[feature_name] = avg_accuracy
    print(f"Average Accuracy dropping '{feature_name}':", avg_accuracy)

# Print overall results
print("Feature Drop Accuracies:", feature_drop_accuracies)


Average Accuracy dropping 'fed_party': 0.7608047138047139
Average Accuracy dropping 'potus_party': 0.7447171717171717
Average Accuracy dropping 'recess': 0.7534175084175084
Average Accuracy dropping 'mom': 0.7231683501683503
Average Accuracy dropping 'pce': 0.763888888888889
Average Accuracy dropping 'ue': 0.7665521885521887
Average Accuracy dropping 'cars': 0.7761010101010102
Average Accuracy dropping 'house': 0.7695084175084175
Average Accuracy dropping 'cli': 0.7819090909090909
Average Accuracy dropping 'exports': 0.7817542087542088
Average Accuracy dropping 'rgdp': 0.7745521885521887
Average Accuracy dropping 'gdpd': 0.7637508417508418
Average Accuracy dropping 'veloc': 0.7805353535353536
Average Accuracy dropping 'ffr': 0.7824814814814814
Average Accuracy dropping 'mich': 0.7782693602693603
Average Accuracy dropping 'd_pce': 0.7841616161616162
Average Accuracy dropping 'd_ue': 0.7866666666666667
Average Accuracy dropping 'd_cars': 0.7835454545454545
Average Accuracy dropping 'd_ho

In [29]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

# Load your data
X, y = load_data_cv('beta_dates/beta_data_2_42.csv', 'beta_dates/true_labels.csv')

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Instantiate and run the TPOT classifier
tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2, random_state=42)
tpot.fit(X_train, y_train)

# Export the best pipeline
tpot.export('best_pipeline.py')

# Evaluate the final model
print("Test score:", tpot.score(X_test, y_test))


                                                                             
Generation 1 - Current best internal CV score: 0.7898520084566596
                                                                              
Generation 2 - Current best internal CV score: 0.7942917547568711
                                                                              
Generation 3 - Current best internal CV score: 0.7942917547568711
                                                                              
Generation 4 - Current best internal CV score: 0.7942917547568711
                                                                              
Generation 5 - Current best internal CV score: 0.8081395348837208
                                                                              
Best pipeline: RandomForestClassifier(input_matrix, bootstrap=True, criterion=gini, max_features=0.55, min_samples_leaf=5, min_samples_split=6, n_estimators=100)
Test score: 0.7454545454545455


In [6]:
from tpot import TPOTClassifier
from sklearn.model_selection import StratifiedKFold

# Load your data
X, y = load_data_cv('beta_dates/beta_data_2_42.csv', 'beta_dates/true_labels.csv')

# Specify K-Fold Cross-Validation
n_splits = 5
cv = StratifiedKFold(n_splits=n_splits, shuffle=True)

# Instantiate and run the TPOT classifier with K-Fold CV
tpot = TPOTClassifier(cv=cv, n_jobs=-1)
tpot.fit(X, y)

# Export the best pipeline
tpot.export('new_best_pipeline.py')

# Evaluate the final model using the test score function
# Note: The test score function will perform a separate train-test split internally
print("Test score:", tpot.score(X, y))


Test score: 0.9708029197080292


In [195]:
from new_best_pipeline import exported_pipeline as new_model
X, y = load_data_cv('beta_dates/beta_data_2_42.csv', 'beta_dates/true_labels.csv')

# split the data into training and test sets
_, X_test, _, y_test = train_test_split(X, y, test_size=0.2)

# predict the labels on the test set
predictions = new_model.predict(X_test)
# Evaluate the model
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       1.00      0.43      0.60         7
         1.0       0.80      0.97      0.88        37
         2.0       0.86      0.55      0.67        11

    accuracy                           0.82        55
   macro avg       0.89      0.65      0.71        55
weighted avg       0.84      0.82      0.80        55



In [344]:
from tpot import TPOTClassifier
from sklearn.model_selection import StratifiedKFold

# Load your data
data = pd.read_csv('famd_data.csv', index_col=0)
y = data['target']
X = data.drop(columns=['target'])

# Specify K-Fold Cross-Validation
n_splits = 5
cv = StratifiedKFold(n_splits=n_splits, shuffle=True)

# Instantiate and run the TPOT classifier with K-Fold CV
tpot = TPOTClassifier(cv=cv, n_jobs=-1)
tpot.fit(X, y)

# Export the best pipeline
tpot.export('famd_pipeline.py')

# Evaluate the final model using the test score function
# Note: The test score function will perform a separate train-test split internally
print("Test score:", tpot.score(X, y))


In [335]:
X = pd.read_csv('famd_data.csv', index_col=0)
y = X['target']
X = X.drop(columns=['target'])

# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_famd, y, test_size=0.2)

# predict the labels on the test set
predictions = tpot.predict(X_test)
# Evaluate the model
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         0.0       1.00      0.56      0.71         9
         1.0       0.85      0.94      0.89        31
         2.0       0.81      0.87      0.84        15

    accuracy                           0.85        55
   macro avg       0.89      0.79      0.82        55
weighted avg       0.87      0.85      0.85        55



In [341]:
# Import the exported pipeline
from famd_best_pipeline import exported_pipeline as famd_model

X = pd.read_csv('famd_data.csv', index_col=0)
y = X['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

predictions = famd_model.predict(X_test)

# split the data into training and test sets

# predict the labels on the test set
# Evaluate the model
print(classification_report(y_test, predictions))


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- target


In [35]:
tpot_params = {'rf__bootstrap': True, 'rf__criterion': 'gini', 'rf__max_depth': 3, 'rf__max_features': 0.15000000000000002, 'rf__min_samples_leaf': 6, 'rf__min_samples_split': 16, 'rf__n_estimators': 100}

# Number of splits for K-Fold Cross-Validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True)

X, y = load_data_cv('beta_dates/beta_data_2_42.csv', 'beta_dates/true_labels.csv')

famd = FAMD(n_components=8)
famd.fit(X)
X = famd.transform(X)

# create a pipeline with the best parameters
pipe = Pipeline([
    ('rf', RandomForestClassifier()),
])

pipe.set_params(**tpot_params)

# Perform K-Fold Cross-Validation

accuracy_scores = []
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Fit the pipeline to the training data
    pipe.fit(X_train, y_train)

    # Make predictions on the test set
    predictions = pipe.predict(X_test)

    # Evaluate the model
    print(classification_report(y_test, predictions))
    print("Accuracy:", accuracy_score(y_test, predictions))
    accuracy_scores.append(accuracy_score(y_test, predictions))

print("Average Accuracy:", np.mean(accuracy_scores))

/opt/homebrew/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         7
         1.0       0.72      1.00      0.84        38
         2.0       1.00      0.20      0.33        10

    accuracy                           0.73        55
   macro avg       0.57      0.40      0.39        55
weighted avg       0.68      0.73      0.64        55

Accuracy: 0.7272727272727273
              precision    recall  f1-score   support

         0.0       1.00      0.17      0.29         6
         1.0       0.72      1.00      0.84        38
         2.0       1.00      0.09      0.17        11

    accuracy                           0.73        55
   macro avg       0.91      0.42      0.43        55
weighted avg       0.80      0.73      0.64        55

Accuracy: 0.7272727272727273
              precision    recall  f1-score   support

         0.0       1.00      0.17      0.29         6
         1.0       0.71      0.97      0.82        38
         2.0     

/opt/homebrew/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [40]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler
from tpot.builtins import ZeroCount

training_features, testing_features, training_target, testing_target = \
            train_test_split(X, y, random_state=None)

# Average CV score on the training set was: 0.8176430976430977
exported_pipeline = make_pipeline(
    MaxAbsScaler(),
    ZeroCount(),
    RandomForestClassifier(bootstrap=False, criterion="entropy", max_features=0.2, min_samples_leaf=4, min_samples_split=5, n_estimators=100)
)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)

accuracy_score(testing_target, results)

0.782608695652174